In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
from glob import glob
import pydicom
import tensorflow as tf
import tqdm as tqdm
import tensorflow_io as tfio
"""
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
"""
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

"\nfor dirname, _, filenames in os.walk('/kaggle/input'):\n    for filename in filenames:\n        print(os.path.join(dirname, filename))\n"

# Creating the Dataset #

In [2]:
class cl_CreatingDataset:
    def __init__(self, imageHeight, imageWidth, batch_size):
        self.imageHeight = imageHeight
        self.imageWidth = imageWidth
        self.batch_size = batch_size

    
    def creatingPathList(self, trainImagesPath, trainDfPath):
        trainDf = pd.read_csv(trainDfPath)
        trainImagesPathList = []
        labels = []
        # reading the tain CSV
        trainDf = pd.read_csv(trainDfPath)
        for i in tqdm.tqdm(range(len(trainDf))):
            folderName = trainDf["StudyInstanceUID"].iloc[i]
            imageFolderPath = os.path.join(trainImagesPath, folderName)

            for file in glob(os.path.join(imageFolderPath, "*.dcm")):
                # taking the imageName
                trainImagesPathList.append(file)
                # creating the labels
                label = np.array([trainDf["C1"].iloc[i],trainDf["C2"].iloc[i], trainDf["C3"].iloc[i], trainDf["C4"].iloc[i], trainDf["C5"].iloc[i], 
                                 trainDf["C6"].iloc[i], trainDf["C7"].iloc[i], trainDf["patient_overall"].iloc[i]])
                labels.append(label)
        
        return trainImagesPathList, labels
    
    
    def parse_function(self, filename, label):
        image_bytes = tf.io.read_file(filename)
        image = tfio.image.decode_dicom_image(image_bytes, dtype=tf.float32)
        #image = tf.image.convert_image_dtype(image, tf.float32)
        resized_image = tf.image.resize(image, [self.imageHeight, self.imageWidth])
        compressedImage = tf.squeeze(resized_image, axis = 0)
        finalImage = tf.repeat(compressedImage, repeats=[3], axis = 2)
        return finalImage, label
        
    def train_preprocess(self, image, label):
        return image, label


    def creatingDataset(self, filenames, labels):
        dataset = tf.data.Dataset.from_tensor_slices((filenames, labels))
        dataset = dataset.shuffle(len(filenames))
        dataset = dataset.map(self.parse_function, num_parallel_calls=4)
        dataset = dataset.map(self.train_preprocess, num_parallel_calls=4)
        dataset = dataset.batch(self.batch_size)
        dataset = dataset.prefetch(1)

        return dataset

## Splitting the dataset ##

In [3]:
def splitDataset(dataset, trainFactor): # here it refers to tf.dataset
    train_dataset = dataset.take(int(trainFactor * len(dataset)))
    validation_dataset = dataset.take(int((1 - trainFactor)* len(dataset)))
    
    return train_dataset, validation_dataset
    

## Creating the model ##

In [4]:
def creating_model():
    IMG_SHAPE = (224, 224, 3)
    base_model = tf.keras.applications.EfficientNetB5(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')
    
    inputs = tf.keras.Input(shape=IMG_SHAPE)
    x = base_model(x, training=False)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    outputs = tf.keras.layers.Dense(8, activation = 'sigmoid')(x)
    model = tf.keras.Model(inputs, outputs)


In [5]:
trainCsv = "../input/rsna-2022-cervical-spine-fracture-detection/train.csv"
trainImagePath = "../input/rsna-2022-cervical-spine-fracture-detection/train_images"
creDataset = cl_CreatingDataset(64, 64, 4)
trainImagesPathList, labels = creDataset.creatingPathList(trainImagePath, trainCsv)
print("train Image list :", len(trainImagePath), ".....", "train label list : ", len(labels))

100%|██████████| 2019/2019 [04:03<00:00,  8.31it/s]

train Image list : 65 ..... train label list :  711601


In [6]:
dataset = creDataset.creatingDataset(trainImagesPathList, labels)

2022-08-09 17:01:45.913042: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2022-08-09 17:02:05.057313: I tensorflow_io/core/kernels/cpu_check.cc:128] Your CPU supports instructions that this TensorFlow IO binary was not compiled to use: AVX2 FMA


In [7]:
for image, label in dataset.take(1):
    imageNp = image.numpy()
    print(imageNp.shape)

2022-08-09 17:02:06.031353: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
W: DcmMetaInfo: No Group Length available in Meta Information Header
W: DcmMetaInfo: No Group Length available in Meta Information Header
W: DcmMetaInfo: No Group Length available in Meta Information Header
W: DcmMetaInfo: No Group Length available in Meta Information Header
W: DcmMetaInfo: No Group Length available in Meta Information Header
W: DcmMetaInfo: No Group Length available in Meta Information Header
W: DcmMetaInfo: No Group Length available in Meta Information Header
W: DcmMetaInfo: No Group Length available in Meta Information Header


(4, 64, 64, 3)
